In [173]:
from pyspark.sql import HiveContext
from pyspark import SparkContext
from pyspark.sql.functions import *   #importar todas as funções do pyspark sql
from pyspark.sql.types import *       #importar as conversoes
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [174]:
sqlContext = HiveContext(sc)

In [175]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|       covid|
|     default|
+------------+



In [176]:
sqlContext.sql("use covid")

DataFrame[]

In [177]:
sqlContext.sql("show tables").show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|   covid|            covid|      false|
|   covid|   covidfirstview|      false|
|   covid|covid_partitioned|      false|
+--------+-----------------+-----------+



In [178]:
covid = sqlContext.sql('select * from covid_partitioned')

In [179]:
covidDF = spark.read.table('covid_partitioned')
covidDF.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codregiaosaude: integer (nullable = true)
 |-- nomeregiaosaude: string (nullable = true)
 |-- data: string (nullable = true)
 |-- semanaepi: integer (nullable = true)
 |-- populacaotcu219: integer (nullable = true)
 |-- casosacumulados: integer (nullable = true)
 |-- casosnovos: integer (nullable = true)
 |-- obitosacumulados: integer (nullable = true)
 |-- obitosnovos: integer (nullable = true)
 |-- recuperadosnovos: integer (nullable = true)
 |-- emacompanhamentonovos: integer (nullable = true)
 |-- interiormetropolitana: integer (nullable = true)
 |-- municipio: string (nullable = true)



In [228]:
covidFirstDF = covidDF.agg((max("casosnovos")-max("emacompanhamentonovos")).alias("Emacompanhamento"), max("emacompanhamentonovos").alias("CasosRecuperados"))
covidFirstDF.show()

+----------------+----------------+
|Emacompanhamento|CasosRecuperados|
+----------------+----------------+
|         1592369|        17262646|
+----------------+----------------+



In [181]:
covidFirstDF.write.saveAsTable("covidFirstView")

AnalysisException: 'Table `covidFirstView` already exists.;'

In [230]:
covidSecDF = covidDF.agg(max("casosnovos").alias("Acumulado"), (max("obitosnovos")-last("casosnovos")).alias("CasosNovos"), format_number((max("casosnovos")*max("populacaotcu219")/100000),2).alias("Incidencia"))
covidSecDF.show()

+---------+----------+----------+
|Acumulado|CasosNovos|Incidencia|
+---------+----------+----------+
| 18855015|     92184|  9,993.16|
+---------+----------+----------+



In [217]:
covidSecDF.write.parquet('/user/marcel/data/covid/covidSecView_teste', compression="snappy")

AnalysisException: 'path hdfs://namenode:8020/user/marcel/data/covid/covidSecView_teste already exists.;'

In [234]:
covidThDF= covidDF.agg(max("obitosnovos").alias("ObitosAcumulados"), (last("obitosacumulados")).alias("CasosNovos"), format_number((max("obitosnovos")/max("casosnovos"))*100,2).alias("Letalidade") ,format_number((max("obitosnovos")*max("populacaotcu219")/100000),2).alias("Mortalidade"))
covidThDF.show()

+----------------+----------+----------+-----------+
|ObitosAcumulados|CasosNovos|Letalidade|Mortalidade|
+----------------+----------+----------+-----------+
|          526892|       594|      2.79|     279.25|
+----------------+----------+----------+-----------+



In [264]:
#covidThDF.write.json('/user/marcel/data/covid/covidThView1')
covidThDF.select(from_json("covidThView1").writeStream)

TypeError: from_json() missing 1 required positional argument: 'schema'